In [1]:
import pandas as pd
import keras
import keras_metrics
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

x_train = pd.read_csv('x_train.csv')
y_train = pd.read_csv('y_train.csv')
x_test = pd.read_csv('x_test.csv')
y_test = pd.read_csv('y_test.csv')

In [2]:
# Training the tokenizer and using that tokenizer to convert a list of tokens to a list of numbers
# Instantiating and fit the tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train['cleanned_text'])

# Use the tokenizer to transform the text messages in the training and test sets
x_train_seq = tokenizer.texts_to_sequences(x_train['cleanned_text'])
x_test_seq = tokenizer.texts_to_sequences(x_test['cleanned_text'])

In [3]:
# Padding each sequesne to the same length, '50' here: Padding the one less than 50 to 50 with 0s
# and truncating the longer ones down to 50
x_train_seq_padded = pad_sequences(x_train_seq, 50)
x_test_seq_padded = pad_sequences(x_test_seq, 50)

In [4]:
# Constructing a simple RNN model
model = Sequential()
model.add(Embedding(len(tokenizer.index_word)+1, 32))
model.add(LSTM(32, dropout = 0, recurrent_dropout = 0))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          264128    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 273,537
Trainable params: 273,537
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Compiling the model
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy', keras_metrics.precision(), keras_metrics.recall()])

In [6]:
import warnings
warnings.filterwarnings("ignore")
# Fit the model
history = model.fit(x_train_seq_padded, y_train,
                    batch_size = 32, epochs = 10,
                    validation_data = (x_test_seq_padded, y_test))

Epoch 1/10
140/140 [==============================] - 13s 63ms/step - loss: 0.4512 - accuracy: 0.8443 - precision: 0.1635 - recall: 0.1354 - val_loss: 0.0885 - val_accuracy: 0.9785 - val_precision: 0.9942 - val_recall: 0.8241
Epoch 2/10
140/140 [==============================] - 4s 31ms/step - loss: 0.0358 - accuracy: 0.9908 - precision: 0.9889 - recall: 0.9492 - val_loss: 0.0705 - val_accuracy: 0.9821 - val_precision: 0.9994 - val_recall: 0.8641
Epoch 3/10
140/140 [==============================] - 4s 30ms/step - loss: 0.0112 - accuracy: 0.9964 - precision: 0.9853 - recall: 0.9913 - val_loss: 0.0603 - val_accuracy: 0.9848 - val_precision: 0.9994 - val_recall: 0.8892
Epoch 4/10
140/140 [==============================] - 4s 30ms/step - loss: 0.0043 - accuracy: 0.9993 - precision: 0.9997 - recall: 0.9972 - val_loss: 0.0713 - val_accuracy: 0.9874 - val_precision: 0.9994 - val_recall: 0.9040
Epoch 5/10
140/140 [==============================] - 4s 29ms/step - loss: 4.0275e-04 - accuracy: 1